## Applying Energy Path algtm to road photos

In [1]:
import cv2
import numpy as np
import matplotlib as plt

In [2]:
RED = (0, 0, 255)

In [40]:
img = cv2.imread("C:\Programs/facultate/licenta/my-autonomous-car/resources/fullCurve.png")


In [41]:
print(img)

[[[207 212 209]
  [220 225 222]
  [224 230 225]
  ...
  [104 109 108]
  [107 111 111]
  [111 116 115]]

 [[201 207 203]
  [224 230 226]
  [222 228 223]
  ...
  [175 180 179]
  [175 180 179]
  [174 179 178]]

 [[181 187 184]
  [227 232 229]
  [224 230 225]
  ...
  [185 189 189]
  [183 188 187]
  [181 186 185]]

 ...

 [[ 66  70  64]
  [ 66  70  64]
  [ 66  70  64]
  ...
  [ 56  60  56]
  [ 56  60  55]
  [ 56  60  55]]

 [[ 66  70  64]
  [ 66  70  64]
  [ 66  70  64]
  ...
  [ 56  60  56]
  [ 56  60  55]
  [ 56  60  55]]

 [[ 66  70  64]
  [ 66  70  64]
  [ 66  70  64]
  ...
  [ 56  60  56]
  [ 56  60  55]
  [ 56  60  55]]]


In [42]:

cv2.imshow("original",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("gray",gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
blurred = cv2.GaussianBlur(gray,(3,3),3)

cv2.imshow("blurred",blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Classic binary thresholding

In [8]:
_, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
# cv2.imwrite("classicTresh.jpg",threshClassis)
cv2.imshow("classic thresh",threshClassis)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Adaptive binary thresholding

In [9]:
threshAdap = cv2.adaptiveThreshold(blurred, 255,
	cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 5, 5)

cv2.imshow("adaptive thresh",threshAdap)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Morphological ops

In [10]:
kernel = np.ones((3,3),np.uint8)
closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)
cv2.imshow("closed img",closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)
cv2.imshow("opened img",opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
_, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
# kernel = np.ones((3,3),np.uint8)
kernel = cv2.getGaussianKernel(5,5)
closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)

# cv2.imwrite("classicTresh.jpg",threshClassis)
cv2.imshow("classic thresh",threshClassis)
cv2.imshow("opened img",opening)
cv2.imshow("closed img",closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Gradient making

In [13]:
def compute_energy(img):
    """
    calculeaza energia la fiecare pixel pe baza gradientului
    :param img: imaginea initiala
    :return:E - energia
    """
    # urmati urmatorii pasi:
    # 1. transformati imagine in grayscale
    # 2. folositi filtru sobel pentru a calcula gradientul in directia X si Y
    # 3. calculati magnitudinea pentru fiecare pixel al imaginii
    E = np.zeros((img.shape[0],img.shape[1]))
    # img_grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    grad_x = cv2.Sobel(img,ddepth=cv2.CV_16S,dx=1,dy=0)
    grad_y = cv2.Sobel(img,ddepth=cv2.CV_16S,dx=0,dy=1)

    abs_x = np.abs(grad_x)
    abs_y = np.abs(grad_y)

    E = abs_x + abs_y

    return E

In [14]:
gradient = compute_energy(opening)
cv2.imshow("gradient img",gradient)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
def select_dynamic_programming_path_going_up(E):
    M = np.zeros(E.shape)
    M[0,:] = E[0,:]
    for i in range(1,E.shape[0]):
        for j  in range (E.shape[1]):
            if j == 0:
                M[i][j] = E[i,j]+max(M[i-1,0],M[i-1,1])
            elif j == E.shape[1] - 1:
                M[i,j] = E[i,j]+max(M[i-1,j],M[i-1,j-1])
            else:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1],M[i][j+1],M[i][j-1])
    line = M.shape[0] - 1
    col = np.argmax(M[line,:])
    print(np.max(M[line,:]))
    path = [0 for i in range(line+1)]

    path[line] = (line,col)
    
    for line in range(M.shape[0]-2,-1,-1):
        if col == 0:
            if M[line,0] < M[line,1]:
                new_col = 1
            else:
                new_col = 0
        elif col == E.shape[1] - 1:
            if M[line,col] < M[line,col-1]:
                new_col = col - 1
            else:
                new_col = col
        else:
            neigh = np.array([M[line,col-1],M[line,col],M[line,col+1]])
            new_col = col+np.argmax(neigh) - 1

        path[line]= (line,new_col)

        col = new_col

    return path

In [33]:
def select_dynamic_programming_path_going_sideways(E):
    M = np.zeros(E.shape)
    M[0,:] = E[0,:]
    for i in range(1,E.shape[0]):
        for j  in range (E.shape[1]):
            if j == 0:
                M[i][j] = E[i,j]+max(M[i-1,0],M[i-1,1])
            elif j == E.shape[1] - 1:
                M[i,j] = E[i,j]+max(M[i-1,j],M[i-1,j-1])
            else:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1],M[i][j+1],M[i][j-1])
    line = M.shape[0] - 1
    col = np.argmax(M[line,:])
    print(np.max(M[line,:]))
    path = [0 for _ in range(60)]
    # path = []
    index = 0

    path[index] = (line,col)

    lineIndex = M.shape[0]-1
    nextLine = False
    while lineIndex > 0 and col >= 0 and col < M.shape[1] and index != 50:
        if col == 0:
            # if M[lineIndex,0] < M[lineIndex,1]:
            #     new_col = 1
            #     nextLine = False
            # else:
            #     new_col = 0
            #     nextLine = False
            if M[lineIndex-1,0] < M[lineIndex-1,1]:
                new_col = 1
                nextLine = True
            else:
                new_col = 0
                nextLine = True
        elif col == E.shape[1] - 1:
            if M[lineIndex-1,col] < M[lineIndex-1,col-1]:
                new_col = col - 1
                nextLine = True
            else:
                new_col = col
                nextLine = True
        else:
            same_line = np.array([M[lineIndex,col-1],M[lineIndex,col+1]])
            upper_line = np.array([M[lineIndex-1,col-1],M[lineIndex-1,col],M[lineIndex-1,col+1]])
            if np.max(same_line) > np.max(upper_line):
                new_col = col+np.argmax(same_line) - 1
                nextLine = False
            else:
                new_col = col+np.argmax(upper_line) - 1
                nextLine = True

        if nextLine:
            lineIndex -= 1

        index += 1  
        path[index]= (lineIndex,new_col)

        col = new_col
    
    return path

In [17]:
def create_path(img, pathL,pathR, color):
    
    new_image = img.copy()

    for row, col in pathL:
        new_image[row, col] = color

    for row, col in pathR:
        new_image[row, col] = color

    # E = compute_energy(img)
    # new_image_E = img.copy()
    # new_image_E[:,:,0] = E.copy()
    # new_image_E[:,:,1] = E.copy()
    # new_image_E[:,:,2] = E.copy()

    # for row, col in path:
    #     new_image_E[row, col] = color
    # cv2.imshow(name, np.uint8(new_image))
    return new_image
    # cv2.imshow('path E', np.uint8(new_image_E))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [27]:
def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray,(3,3),3)

    _, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)

    return opening


In [ ]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
cv2.imshow('PreR', preR)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Path finding going always up

In [20]:
cv2.imshow("a",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [46]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
gradL = compute_energy(preL)
gradR = compute_energy(preR)
pathL = select_dynamic_programming_path_going_up(gradL)
print(pathL)
pathR = select_dynamic_programming_path_going_up(gradR)
pathR =([(x,np.uint64(y + int(pre.shape[1]/2))) for x,y in pathR])

# cv2.imshow('preprocess img', pre)
cv2.imshow('gradientL img', gradL)
cv2.imshow('gradientR img', gradR)
new_img = create_path(img,pathL,pathR,RED)
cv2.imshow("New Image",new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

345780.0
[(0, np.int64(48)), (1, np.int64(49)), (2, np.int64(50)), (3, np.int64(50)), (4, np.int64(51)), (5, np.int64(51)), (6, np.int64(51)), (7, np.int64(51)), (8, np.int64(51)), (9, np.int64(50)), (10, np.int64(49)), (11, np.int64(48)), (12, np.int64(47)), (13, np.int64(46)), (14, np.int64(45)), (15, np.int64(44)), (16, np.int64(43)), (17, np.int64(42)), (18, np.int64(41)), (19, np.int64(40)), (20, np.int64(39)), (21, np.int64(38)), (22, np.int64(37)), (23, np.int64(36)), (24, np.int64(35)), (25, np.int64(34)), (26, np.int64(33)), (27, np.int64(32)), (28, np.int64(31)), (29, np.int64(30)), (30, np.int64(29)), (31, np.int64(28)), (32, np.int64(27)), (33, np.int64(26)), (34, np.int64(25)), (35, np.int64(24)), (36, np.int64(23)), (37, np.int64(22)), (38, np.int64(21)), (39, np.int64(20)), (40, np.int64(19)), (41, np.int64(18)), (42, np.int64(17)), (43, np.int64(16)), (44, np.int64(15)), (45, np.int64(14)), (46, np.int64(13)), (47, np.int64(12)), (48, np.int64(11)), (49, np.int64(10)), 

### Path finding going sideways as well

In [36]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
gradL = compute_energy(preL)
gradR = compute_energy(preR)
pathL = select_dynamic_programming_path_going_sideways(gradL)
print(pathL)
pathR = select_dynamic_programming_path_going_sideways(gradR)
pathR =([(x,np.uint64(y + int(pre.shape[1]/2))) for x,y in pathR])
# # print(newPathR)
# # cv2.imshow('preprocess img', pre)
# cv2.imshow('gradientL img', gradL)
# cv2.imshow('gradientR img', gradR)
new_img = create_path(img,pathL,pathR,RED)
# cv2.imshow("New Image",new_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

266220.0
[(239, np.int64(0)), (238, 0), (237, 0), (236, 0), (235, 0), (234, 0), (233, 0), (232, 0), (231, 0), (230, 0), (229, 0), (228, 0), (227, 0), (226, 0), (225, 0), (224, 0), (223, 0), (222, 0), (221, 0), (220, 0), (219, 0), (218, 0), (217, 0), (216, 0), (215, 0), (214, 0), (213, 0), (212, 0), (211, 0), (210, 0), (209, 0), (208, 0), (207, 0), (206, 0), (205, 0), (204, 0), (203, 1), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), 0, 0, 0, 0, 0, 0, 0, 0, 0]
555900.0


TypeError: cannot unpack non-iterable int object

In [ ]:
cap = cv2.VideoCapture('C:/Programs/facultate/licenta/my-autonomous-car/resources/1.mp4')
 
while cap.isOpened():
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    pre = preprocess(frame)
    preL = pre[:,:int(pre.shape[1]/2)]
    preR = pre[:,int(pre.shape[1]/2):]
    gradL = compute_energy(preL)
    gradR = compute_energy(preR)
    pathL = select_dynamic_programming_path_going_up(gradL)
    pathR = select_dynamic_programming_path_going_up(gradR)
    pathR =([(x,np.uint64(y + int(pre.shape[1]/2))) for x,y in pathR])
    new_img = create_path(frame,pathL,pathR,RED)
    cv2.imshow("pre",pre)
    cv2.imshow('newImg', new_img)
    
    if cv2.waitKey(1) == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()